## Import Libraries

In [ ]:
from crewai import Agent, Task, Crew
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun, PythonREPLTool
import os


In [ ]:
os.environ['OPENAI_API_KEY'] = 'your-openai-key-here'  # Replace with your key


## Define the Flow State (Shared Data Container)


class MyFlowState(BaseModel):
    data: pd.DataFrame = None
    insights: str = None
    chart_path: str = None
    summary: str = None


## Define the 4 Agents

loader_agent = Agent(
    role='Loader Agent',
    goal='Load and clean the dataset',
    backstory='Expert in data loading',
    verbose=True
)

analyzer_agent = Agent(
    role='Analyzer Agent',
    goal='Extract insights from the data',
    backstory='Specialist in statistical analysis',
    verbose=True
)

visualizer_agent = Agent(
    role='Visualizer Agent',
    goal='Create visual plots from the data',
    backstory='Data visualization expert',
    verbose=True
)

summarizer_agent = Agent(
    role='Summarizer Agent',
    goal='Write a summary of the analysis',
    backstory='Skilled in summarizing findings',
    verbose=True
)


## Define the Flow Logic

class MyFlow(Flow):
    class Config:
        flow_state = MyFlowState

    @start
    def load_data(self):
        # Simulated data
        df = pd.DataFrame({
            'Year': [2020, 2021, 2022, 2023],
            'Revenue': [100, 150, 130, 180],
            'Expenses': [80, 100, 90, 120]
        })
        self.state.data = df
        return 'Data loaded.'

    @listen(load_data)
    def analyze_data(self):
        df = self.state.data
        insight = f"Avg Revenue: {df['Revenue'].mean()}, Avg Expenses: {df['Expenses'].mean()}"
        self.state.insights = insight
        return insight

    @listen(load_data)
    def visualize_data(self):
        df = self.state.data
        plt.figure(figsize=(6, 4))
        plt.plot(df['Year'], df['Revenue'], label='Revenue')
        plt.plot(df['Year'], df['Expenses'], label='Expenses')
        plt.legend()
        plt.title("Revenue vs Expenses")
        chart_path = "/mnt/data/revenue_vs_expenses.png"
        plt.savefig(chart_path)
        self.state.chart_path = chart_path
        return f"Chart saved at {chart_path}"

    @listen(load_data)
    def summarize_data(self):
        df = self.state.data
        summary = f"In {df['Year'].iloc[-1]}, Revenue was {df['Revenue'].iloc[-1]} and Expenses were {df['Expenses'].iloc[-1]}."
        self.state.summary = summary
        return summary


## Run the Flow

flow = MyFlow()
flow_output = flow.kickoff()
flow_output

## Check Final State

flow.state